# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [6]:
#load city data file from part 1 weatherpy
vaca_df = pd.read_csv("output_data/cities_eewimmer.csv")

#check it out
vaca_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Lihue,21.9789,-159.3672,74.79,83,75,7.54,1020,US,1733937947
1,1,Picota,-6.9197,-76.3304,94.26,42,99,7.54,1010,PE,1733937948
2,2,Edinburgh of the Seven Seas,-37.0676,-12.3116,58.48,75,100,7.54,1022,SH,1733937949
3,3,Port-aux-Français,-49.3500,70.2167,41.95,88,92,7.54,981,TF,1733937950
4,4,Dudinka,69.4058,86.1778,8.37,91,100,7.54,997,RU,1733937951


In [8]:
# clean up those dates
vaca_df["Date"] = pd.to_datetime(vaca_df.Date * 1e9)
vaca_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Lihue,21.9789,-159.3672,74.79,83,75,7.54,1020,US,2024-12-11 17:25:47
1,1,Picota,-6.9197,-76.3304,94.26,42,99,7.54,1010,PE,2024-12-11 17:25:48
2,2,Edinburgh of the Seven Seas,-37.0676,-12.3116,58.48,75,100,7.54,1022,SH,2024-12-11 17:25:49
3,3,Port-aux-Français,-49.3500,70.2167,41.95,88,92,7.54,981,TF,2024-12-11 17:25:50
4,4,Dudinka,69.4058,86.1778,8.37,91,100,7.54,997,RU,2024-12-11 17:25:51


In [9]:
vaca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     550 non-null    int64         
 1   City        550 non-null    object        
 2   Lat         550 non-null    float64       
 3   Lng         550 non-null    float64       
 4   Max Temp    550 non-null    float64       
 5   Humidity    550 non-null    int64         
 6   Cloudiness  550 non-null    int64         
 7   Wind Speed  550 non-null    float64       
 8   Pressure    550 non-null    int64         
 9   Country     548 non-null    object        
 10  Date        550 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 47.4+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

#set the map plot with city populating in hover
map_plot = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    size = "Humidity",
    hover_cols=["City", "Country", "Humidity"]
)

#print
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 74
max_temp = 88
max_wind = 10
max_cloudiness = 50
# Drop any rows with null values
warm_vaca = vaca_df.dropna()

mask = (warm_vaca["Max Temp"] >= min_temp) & (warm_vaca["Max Temp"] < max_temp) & (warm_vaca["Wind Speed"] < max_wind) & (warm_vaca["Cloudiness"] < max_cloudiness)
warm_vaca = warm_vaca.loc[mask].reset_index(drop=True)

# Display sample data
warm_vaca

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,15,Nadi,-17.8000,177.4167,75.20,88,40,7.54,1008,FJ,2024-12-11 17:26:06
1,29,Port Blair,11.6667,92.7500,82.44,83,20,7.54,1009,IN,2024-12-11 17:26:22
2,38,Suva,-18.1416,178.4415,74.68,94,40,7.54,1009,FJ,2024-12-11 17:26:33
3,45,Koungou,-12.7336,45.2042,83.91,74,0,7.54,1012,YT,2024-12-11 17:26:42
4,62,Pointe aux Piments,-20.0628,57.5131,75.52,92,28,7.54,1015,MU,2024-12-11 17:27:01
...,...,...,...,...,...,...,...,...,...,...,...
84,523,Al Mijlad,11.0333,27.7333,81.18,19,2,7.54,1010,SD,2024-12-11 17:36:17
85,533,Takoradze,4.8845,-1.7554,80.78,80,12,7.54,1009,GH,2024-12-11 17:36:30
86,537,Akonolinga,3.7667,12.2500,78.76,34,41,7.54,1009,CM,2024-12-11 17:36:34
87,545,Pimentel,-6.8367,-79.9342,75.49,57,0,7.54,1011,PE,2024-12-11 17:36:43


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
#create list
hotel_rows = []

for index, row in warm_vaca.iterrows():
    #from weatherPy .csv
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    humidity = row["Humidity"]

    #find the first hotel within 10,000 m
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    #parameters
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    #parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    #base URL
    base_url = "https://api.geoapify.com/v2/places"

    #run it!
    try:
        response = requests.get(base_url, params=params)
        #print(response.status_code)
        data = response.json()
        
        #print
        results = data.get("features", [])
        
        #deal with potential errors
        if len(results) > 0:
            place = results[0]
    
            #normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            #rename outputs
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    #update to hotel list
    hotel_rows.append(hotel_row)

In [36]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Nadi,FJ,-17.8000,177.4167,None,None,None,None,None
1,Port Blair,IN,11.6667,92.7500,None,None,None,None,None
2,Suva,FJ,-18.1416,178.4415,None,None,None,None,None
3,Koungou,YT,-12.7336,45.2042,None,None,None,None,None
4,Pointe aux Piments,MU,-20.0628,57.5131,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
84,Al Mijlad,SD,11.0333,27.7333,None,None,None,None,None
85,Takoradze,GH,4.8845,-1.7554,None,None,None,None,None
86,Akonolinga,CM,3.7667,12.2500,None,None,None,None,None
87,Pimentel,PE,-6.8367,-79.9342,None,None,None,None,None


In [37]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["longitude", "latitude", "city", "country", "name"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE